# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей вы получите из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составьте первое представление о данных Яндекс.Музыки.




Основной инструмент аналитика — `pandas`. Импортируйте эту библиотеку.

In [1]:
import pandas as pd # импорт библиотеки pandas

Прочитайте файл `yandex_music_project.csv` из папки `/datasets` и сохраните его в переменной `df`:

In [2]:
try:
    df = pd.read_csv('yandex_music_project.csv') # чтение файла с данными и сохранение в df
except:
    df = pd.read_csv('/datasets/yandex_music_project.csv') # если файл лежит не локально, а на сервере yandex и запуск идет в их окружении

Выведите на экран первые десять строк таблицы:

In [3]:
df.head(10)# получение первых 10 строк таблицы df

Одной командой получить общую информацию о таблице:

In [4]:
df.info() # получение общей информации о данных в таблице df

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Поле `userID` после преобразования будет выглядеть как `userid`, что затруднит его восприятие, необходимо применить "змеиный" стиль в названии - `user_id`.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправьте стиль в заголовках столбцов, исключите пропуски. Затем проверьте данные на дубликаты.

### Стиль заголовков
Выведите на экран названия столбцов:

In [5]:
df.columns # перечень названий столбцов таблицы df

Приведите названия в соответствие с хорошим стилем:
* несколько слов в названии запишите в «змеином_регистре»,
* все символы сделайте строчными,
* устраните пробелы.

Для этого переименуйте колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
df.rename(columns = {
    '  userID' : 'user_id', 
    'Track'    : 'track', 
    '  City  ' : 'city', 
    'Day'      : 'day'
},inplace = True) # переименование столбцов

Проверьте результат. Для этого ещё раз выведите на экран названия столбцов:

In [7]:
df.columns # проверка результатов - перечень названий столбцов

### Пропуски значений
Сначала посчитайте, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [8]:
df.isna().sum() # подсчёт пропусков

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Выбран метод isna() для нахождения пропущенных значений, это отлично! На самом деле, isnull() и isna() делают одно и то же, но использование isna() предпочтительнее. </div>

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Замените пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создайте список `columns_to_replace`, переберите его элементы циклом `for` и для каждого столбца выполните замену пропущенных значений:

In [9]:
columns_to_replace = ['track','artist','genre'] # объявляем список со столбцами, в которых есть пропущенные значения
for column in columns_to_replace:      # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
    df[column] = df[column].fillna('unknown')
    

Убедитесь, что в таблице не осталось пропусков. Для этого ещё раз посчитайте пропущенные значения.

In [10]:
df.isna().sum() # подсчёт пропусков

### Дубликаты
Посчитайте явные дубликаты в таблице одной командой:

In [11]:
df.duplicated().sum() # подсчёт явных дубликатов

Вызовите специальный метод `pandas`, чтобы удалить явные дубликаты:

In [12]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

Ещё раз посчитайте явные дубликаты в таблице — убедитесь, что полностью от них избавились:

In [13]:
df.duplicated().sum() # проверка на отсутствие дубликатов

Теперь избавьтесь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведите на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлеките нужный столбец датафрейма, 
* примените к нему метод сортировки,
* для отсортированного столбца вызовите метод, который вернёт уникальные значения из столбца.

In [14]:
df['genre'].sort_values().unique() # Просмотр уникальных названий жанров

Просмотрите список и найдите неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Вы увидите следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишите функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [15]:
def replace_wrong_genres(wrong_genres = [],correct_genre = ''):# Функция для замены неявных дубликатов
    for wr_genre in wrong_genres:                              # для каждого жанра из списка неправильных жанров
        df['genre'] = df['genre'].replace(wr_genre, correct_genre)  # меняем неправильное название на правильное

Вызовите `replace_wrong_genres()` и передайте ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [16]:
wrong_hiphop_list = ['hip', 'hop', 'hip-hop'] # Устранение неявных дубликатов жанра hiphop
replace_wrong_genres(wrong_hiphop_list, 'hiphop')

Проверьте, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведите отсортированный список уникальных значений столбца `genre`:

In [17]:
df['genre'] = df['genre'].replace('электроника', 'electronic') # Устранение неявных дубликатов жанра electronic
df['genre'] = df['genre'].replace('ïîï', 'pop') # Устранение неявных дубликатов жанра pop

In [18]:
df['genre'].sort_values().unique() # Проверка на неявные дубликаты

Проверим, все ли остальные значения в столбцах `city`,`day` и `time` имеют уникальные значения:

In [19]:
print(df['city'].unique()) # Поиск уникальных значений столбца city (город)
print(df['day'].unique())  # Поиск уникальных значений столбца day (день недели)
print(df['time'].sort_values().unique())

Можно сделать 2 вывода по итогу предварительного аналица уникальных значений столбцов `city`, `day` и `time`:
* Названия городов уникальны и их всего два, как и обозначено в условии задачи.
* Данные по прослушиваниям собраны по трём дня недели: понедельник(`'Monday'`), среда (`'Wednesday'`) и пятница(`'Friday'`).
* Данные по времени прослушивания содержат строки в формате `чч:мм:сс` за период с 8 до 22 часов. 

Эта информация может пригодиться в ходе дальнейшего исследования.

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Вы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделите пользователей Москвы и Санкт-Петербурга
* Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполните каждый из расчётов по отдельности. 

Оцените активность пользователей в каждом городе. Сгруппируйте данные по городу и посчитайте прослушивания в каждой группе.



In [20]:
df.groupby('city')['track'].count() # Подсчёт прослушиваний в каждом городе

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируйте данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтите, что в данных есть информация только о прослушиваниях только за эти дни.


In [21]:
print(df.groupby('day')['track'].count()) # Подсчёт прослушиваний в каждый из трёх дней

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Вы видели, как работает группировка по городу и по дням недели. Теперь напишите функцию, которая объединит два эти расчёта.

Создайте функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраните в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого примените последовательную фильтрацию с логической индексацией.

Затем посчитайте значения в столбце `user_id` получившейся таблицы. Результат сохраните в новую переменную. Верните эту переменную из функции.

In [22]:
# <создание функции number_tracks()>
def number_tracks(day = '', city = ''): # Объявляется функция с двумя параметрами: day, city.
    track_list = df[df['day'] == day]   # В переменной track_list сохраняются те строки таблицы df, для которых 
                                        # значение в столбце 'day' равно параметру day и одновременно значение
    track_list = track_list[track_list['city'] == city] # в столбце 'city' равно параметру city
    track_list_count = track_list['user_id'].count()    # track_list_count сохраняется число значений столбца 'user_id'
                                                        # рассчитанное методом count() для таблицы track_list.
    return track_list_count                             # Функция возвращает число - значение track_list_count.

# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

Вызовите `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [23]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

In [24]:
number_tracks('Monday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по понедельникам

In [25]:
number_tracks('Wednesday', 'Moscow') # количество прослушиваний в Москве по средам

In [26]:
number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

In [27]:
number_tracks('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам

In [28]:
number_tracks('Friday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по пятницам

Создайте c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [29]:
columns_names = ['city', 'monday', 'wednesday', 'friday'] # Создаем список с названием столбцов будущей таблицы
data = [['Moscow', 15740, 11056, 15945],                  # Создаем данные о количестве прослушиваний в Москве
        ['Saint-Petersburg', 5614, 7003, 5895]            # и Санкт-Петербурге
       ]

In [30]:
result_df = pd.DataFrame(data, columns = columns_names) # Таблица с результатами
print('Данные о прослушивании музыки по городам и дням недели:')
display(result_df)

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [31]:
moscow_general = df[df['city'] == 'Moscow'] # получение таблицы moscow_general из тех строк таблицы df, 
                                            # для которых значение в столбце 'city' равно 'Moscow'
moscow_general = moscow_general.drop(columns = ['city']) # Удаляем столбец 'city' за ненадобностью



In [32]:
spb_general = df[df['city'] == 'Saint-Petersburg'] # получение таблицы spb_general из тех строк таблицы df,
                                                   # для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = spb_general.drop(columns = ['city']) # Удаляем столбец 'city' за ненадобностью



Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [33]:
def genre_weekday(table, day, time1, time2): # Объявление функ. genre_weekday() с параметрами table, day, time1, time2,
                                           # которая возвращает информацию о самых популярных жанрах в указанный день в
                                           # заданное время:
                                           # в genre_df сохраняются те строки переданного датафрейма table, для
                                           #    которых одновременно:
    genre_df = table[table['day'] == day]  #    - значение в столбце day равно значению аргумента day
    genre_df = genre_df[genre_df['time'] > time1] #    - значение в столбце time больше значения аргумента time1
    genre_df = genre_df[genre_df['time'] < time2] #    - значение в столбце time меньше значения аргумента time2
    genre_df_count = genre_df.groupby(by = genre_df['genre'])['user_id'].count() #считаем методом count() 
                                            #    количество записей для каждого из
                                            #    присутствующих жанров, получившийся Series записать в переменную
                                            #    genre_df_count
    genre_df_sorted = genre_df_count.sort_values(ascending=False) # сортировка genre_df_count по убыванию
                                            #    встречаемости и сохранение в переменную genre_df_sorted
    return genre_df_sorted.head(10)         # возврат Series из 10 первых значений genre_df_sorted, это будут топ-10
                                            #    популярных жанров (в указанный день, в заданное время)

Cравните результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [34]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Москве
# объекты, хранящие время, являются строками и сравниваются как строки

In [35]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Петербурге

In [36]:
genre_weekday(moscow_general, 'Friday', '18:00', '23:00') # вызов функции для вечера пятницы в Москве

In [37]:
genre_weekday(spb_general, 'Friday', '18:00', '23:00')# вызов функции для вечера пятницы в Петербурге

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируйте таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируйте результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [38]:
moscow_genres = moscow_general.groupby('genre')# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
moscow_genres = moscow_genres['user_id'].count()            # подсчёт числа значений 'genre' методом count(), 
moscow_genres = moscow_genres.sort_values(ascending = False)# сортировка получившегося Series в порядке убывания и 
                                                            # сохранение в moscow_genres

Выведите на экран первые десять строк `moscow_genres`:

In [39]:
moscow_genres.head(10) # просмотр первых 10 строк moscow_genres

Теперь повторите то же и для Петербурга.

Сгруппируйте таблицу `spb_general` по жанру. Посчитайте прослушивания треков каждого жанра. Результат отсортируйте в порядке убывания и сохраните в таблице `spb_genres`:


In [40]:
spb_genres = spb_general.groupby('genre') # одной строкой: группировка таблицы spb_general по столбцу 'genre', 
spb_genres = spb_genres['user_id'].count() # подсчёт числа значений 'genre' в этой группировке методом count(), 
spb_genres = spb_genres.sort_values(ascending = False) # сортировка получившегося Series в порядке убывания и 
                                                       # сохранение в spb_genres

Выведите на экран первые десять строк `spb_genres`:

In [41]:
spb_genres.head(10) # просмотр первых 10 строк spb_genres

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.